# 1. Camera Calibration

This script intends to calibrate a camera using an object.\
For this implementation we will use the openCV method for calibration(https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html)

**Import the libraries**

In [47]:
from platform import python_version

print(python_version())

3.8.5


In [43]:
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
import cv2

#for LM method
from scipy.optimize import curve_fit
#matplotlib widget

In [44]:
if False:
    folder_dir = "C:/Users/UX550/OneDrive - Universidade de Aveiro/Documentos/5º ANO\Thesis/calibration_photos/calibration_2_meters_tivera/right_2/"
    img_cnt = 0
    # list of images
    list_images = os.listdir(folder_dir)
    print(list_images)

In [45]:
if False:
    folder_dir = "C:/Users/UX550/OneDrive - Universidade de Aveiro/Documentos/5º ANO\Thesis/calibration_photos/calibration_2_meters_tivera/left_2/"
    img_cnt = 0
    # list of images
    list_images = os.listdir(folder_dir)
    print(list_images)

### Run through image directory

In [3]:

if True:
        
    import os 
    import string
    from os import listdir

    camera_name = 'left'
    # get the path/directory
    #C:/Users/UX550/OneDrive - Universidade de Aveiro/Documentos/5º ANO\Thesis/calibration_photos/calibration_2_meters_tivera/right_2/
    folder_dir = f"C:/Users/UX550\OneDrive - Universidade de Aveiro/Documentos/5º ANO/Thesis/distances/distances_3_4/{camera_name}_3_distances/{camera_name}_3_45/" 
    #folder_dir = f"C:/Users/UX550\OneDrive - Universidade de Aveiro/Documentos/5º ANO/Thesis/distances/{camera_name}_2_5"
    img_cnt = 0
    # list of images
    list_images = os.listdir(folder_dir)
    new_list = sorted(list_images,key = lambda k: int(k.split('_')[2].split('.')[0]))

    for images in new_list:
        # check if the image ends with png
        renamed_image = camera_name + "_init_" + '%s' % img_cnt + '.png'
        print(images)
        print(renamed_image)
        # If equal, no renaming
        if(images == renamed_image):
            print("equal")
        else:
            while True:
                print(images)
                renamed_image = camera_name + "_init_" + '%s' % img_cnt + '.png'
                print(renamed_image)
                try:
                    os.rename(folder_dir + '/' + images,folder_dir + '/' + renamed_image)
                except FileExistsError:
                    img_cnt += 1
                    pass
                else:
                    break
                
            
        img_cnt += 1


if True:
        
    import os 
    import string
    from os import listdir

    camera_name = 'right'
    # get the path/directory
    #C:/Users/UX550/OneDrive - Universidade de Aveiro/Documentos/5º ANO\Thesis/calibration_photos/calibration_2_meters_tivera/right_2/
    folder_dir = f"C:/Users/UX550\OneDrive - Universidade de Aveiro/Documentos/5º ANO/Thesis/distances/distances_3_4/{camera_name}_3_distances/{camera_name}_3_45/" 
    #folder_dir = f"C:/Users/UX550\OneDrive - Universidade de Aveiro/Documentos/5º ANO/Thesis/distances/{camera_name}_2_5"
    img_cnt = 0
    # list of images
    list_images = os.listdir(folder_dir)
    new_list = sorted(list_images,key = lambda k: int(k.split('_')[2].split('.')[0]))

    for images in new_list:
        # check if the image ends with png
        renamed_image = camera_name + "_init_" + '%s' % img_cnt + '.png'
        print(images)
        print(renamed_image)
        # If equal, no renaming
        if(images == renamed_image):
            print("equal")
        else:
            while True:
                print(images)
                renamed_image = camera_name + "_init_" + '%s' % img_cnt + '.png'
                print(renamed_image)
                try:
                    os.rename(folder_dir + '/' + images,folder_dir + '/' + renamed_image)
                except FileExistsError:
                    img_cnt += 1
                    pass
                else:
                    break
                
            
        img_cnt += 1


left_init_6.png
left_init_0.png
left_init_6.png
left_init_0.png
left_init_9.png
left_init_1.png
left_init_9.png
left_init_1.png
left_init_13.png
left_init_2.png
left_init_13.png
left_init_2.png
left_init_19.png
left_init_3.png
left_init_19.png
left_init_3.png
left_init_22.png
left_init_4.png
left_init_22.png
left_init_4.png
left_init_24.png
left_init_5.png
left_init_24.png
left_init_5.png
left_init_28.png
left_init_6.png
left_init_28.png
left_init_6.png
left_init_32.png
left_init_7.png
left_init_32.png
left_init_7.png
left_init_35.png
left_init_8.png
left_init_35.png
left_init_8.png
left_init_39.png
left_init_9.png
left_init_39.png
left_init_9.png
right_init_6.png
right_init_0.png
right_init_6.png
right_init_0.png
right_init_9.png
right_init_1.png
right_init_9.png
right_init_1.png
right_init_13.png
right_init_2.png
right_init_13.png
right_init_2.png
right_init_19.png
right_init_3.png
right_init_19.png
right_init_3.png
right_init_22.png
right_init_4.png
right_init_22.png
right_init_4.pn

## Zhang's Method
Comparing to the DLT method, the Zhang method only calculate the intrinsic parameters. We instead only need to find 5 parameters.

## 2. Calibration

## Find the homographies

In [55]:
def cornerfinding(n_imgs,patternsize,dist, img_name):
    '''
    @param
    [in] n_imgs : number of images used
    [in] patternsize : size of the checkerboard
    [in] dist : distance between squares
    [in] img_name : name of the jpg
    [out] imagecoo : imagecoordinates
    [out] worldcoo : worldcoordinates
    '''
    # number of images
    n_imgs = n_imgs
    # pattern size
    patternsize= patternsize
    #distance between squares(in mm)
    dist = dist

    # specified number of iterations are completed.
    criteria = (cv2.TERM_CRITERIA_EPS +
                cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Vector for 3D points
    threedpoints = []

    # Vector for 2D points
    twodpoints = []

    #3D points real world coordinates( static -> chessboard)
    # create a matrix of zeros for 48 points , each with 3 coordinates
    world_coo = np.zeros((1, patternsize[0]* patternsize[1],3), np.float32)
    world_coo[:, :, :2] = np.mgrid[0:patternsize[0], 0:patternsize[1]].T.reshape(-1, 2)
    # Transform coordinates in mm
    world_coo = world_coo*dist

    #prev_img_shape = None

    for i in range(n_imgs):

        im = i

        #Import the image
        name = (str(img_name)+ str(im)+".png")
        img = cv2.imread(name)
        #print(name)
        # convert it to gray scale
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        retval, corners= cv2.findChessboardCorners(gray,patternsize,None)
        #print(corners)
        if(retval == True):
            threedpoints.append(world_coo)
            #print(name)

            # Refining pixel coordinates or given 2d points.
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria).reshape(patternsize[0]*patternsize[1],2)

            twodpoints.append(corners2)
            # Draw and display the corners
            image = cv2.drawChessboardCorners(img,patternsize,corners2,retval)
            #plt.imshow(image)
            #plt.show()
        else:
            print(name)
            os.remove(name)
            other_name = ("../Thesis/calibration_photos/calibration_3_noinfi/left_distances_2/left_init_")+ str(im)+".png"
            os.remove(other_name)

    # image coordinates and world coordinates for the diferent images( the world coordinates are always the same)
    imagecoo = np.asarray(twodpoints)
    worldcoo = np.asarray(threedpoints) 
    return imagecoo, worldcoo

#C:\Users\UX550\OneDrive - Universidade de Aveiro\Documentos\5º ANO\Thesis\calibration\images
#name_left = ("../calibration_photos/calibration_photos_2meters/left_160/left_init_")
#name_right =("../calibration_photos/calibration_photos_2meters/right_160/right_init_")
#name_left = ("../calibration_photos/calibration_photos/left_edited/left_init_")
#name_right =("../calibration_photos/calibration_photos/right_edited/right_init_")
#name_left = "../Thesis/calibration_photos/calibration/left_3/left_init_"
#name_right = ("../Thesis/calibration_photos/calibration/right_3/right_init_")
name_left = ("../Thesis/calibration_photos/calibration_3_noinfi/left_distances_2/left_init_")
name_right = ("../Thesis/calibration_photos/calibration_3_noinfi/right_distances_2/right_init_")
#name_right = ("../calibration_photos/calibration_2_meters_tivera/right_2/right_init_")
#name_left = ("calibration_photos/left_edited/left_init_")
#name_right = ("calibration_photos/right_edited/right_init_")
n_imgs = 125
square_size = 44
pattern_size = (12,8)
imagecoo_left,worldcoo_left = cornerfinding(n_imgs,pattern_size,square_size,name_left)
imagecoo_right,worldcoo_right = cornerfinding(n_imgs,pattern_size,square_size,name_right)


In [56]:
def calib(world_pts_arr,x_zhang_R,x_zhang_L,image_size):    
    ret_R, mtx_R, dist_R, rvecs_R, tvecs_R = cv2.calibrateCamera(world_pts_arr, x_zhang_R, (image_size[0],image_size[1]), None, None)
    ret_L, mtx_L, dist_L, rvecs_L, tvecs_L = cv2.calibrateCamera(world_pts_arr, x_zhang_L, (image_size[0],image_size[1]), None, None)



    flags = 0
    flags |= cv2.CALIB_FIX_INTRINSIC
    # Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
    # Hence intrinsic parameters are the same 

    criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


    # This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
    retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(world_pts_arr, x_zhang_L, x_zhang_R, mtx_L, dist_L, mtx_R, dist_R, (image_size[0],image_size[1]), criteria_stereo, flags)

    return ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat,distL,distR,rvecs_L,rvecs_R

In [57]:
worldcoo_left.shape

(125, 1, 96, 3)

In [58]:
type(imagecoo_right[0,0,0])

numpy.float32

In [59]:
#image_size = (1936,1216)
#ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat,dist_L,dist_R,rvecs_L,rvecs_R = calib(worldcoo_left,imagecoo_right,imagecoo_left,image_size)
#retS


In [60]:
image_size = (1936,1216)


ret_R_list = []
mtx_R_list = []
ret_L_list = []
mtx_L_list = []
retS_list = []
new_mtxL_list = []
new_mtxR_list = []
Rot_list = []
Trns_list = []
Emat_list = []
Fmat_list = []
P_R_org_list = []
P_L_org_list = []
dist_L_list = []
dist_R_list = []
rvecs_L_list = []
rvecs_R_list = []

ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat,dist_L,dist_R,rvecs_L,rvecs_R = calib(worldcoo_left,imagecoo_right,imagecoo_left,image_size)


ret_R_list.append([ret_R])
mtx_R_list.append([mtx_R])
ret_L_list.append([ret_L])
mtx_L_list.append([mtx_L])
retS_list.append([retS])
new_mtxL_list.append([new_mtxL])
new_mtxR_list.append([new_mtxR])
Rot_list.append([Rot])
Trns_list.append([Trns])
Emat_list.append([Emat])
Fmat_list.append([Fmat])
dist_L_list.append(dist_L)
dist_R_list.append(dist_R)
rvecs_L_list.append(rvecs_L)
rvecs_R_list.append(rvecs_R)

In [61]:
P_L_list = []
P_R_list = []
R1_list = []
R2_list = []
R1,R2,P_L_est,P_R_est,Q,roi_left,roi_right = cv2.stereoRectify(new_mtxL, ret_L, new_mtxR, ret_R, (image_size[0],image_size[1]), Rot, Trns,flags = cv2.CALIB_ZERO_DISPARITY)

R1_list.append(R1)
R2_list.append(R2)
P_L_list.append(P_L_est)
P_R_list.append(P_R_est)

In [62]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [63]:
R2

array([[0.991409, 0.010093, -0.130405],
       [-0.010891, 0.999926, -0.005403],
       [0.130341, 0.006776, 0.991446]])

In [64]:
%store R1,R2,retS_list,P_L_est,new_mtxL_list,ret_L_list,dist_L_list,P_R_est,new_mtxR_list,ret_R_list,dist_R_list,Rot_list,Trns_list >".\calibration_files\3_meters\3_meters_noinfi\calib_3_{n_imgs}.txt"

Writing 'R1,R2,retS_list,P_L_est,new_mtxL_list,ret_L_list,dist_L_list,P_R_est,new_mtxR_list,ret_R_list,dist_R_list,Rot_list,Trns_list' (tuple) to file '.\calibration_files\3_meters\3_meters_noinfi\calib_3_125.txt'.


In [ ]:
new_mtxL

array([[3177.394344, 0.000000, 868.514520],
       [0.000000, 3193.865542, 613.104839],
       [0.000000, 0.000000, 1.000000]])

In [ ]:
new_mtxR

array([[2740.523940, 0.000000, 1011.343989],
       [0.000000, 2745.916176, 673.805539],
       [0.000000, 0.000000, 1.000000]])

In [ ]:
P_R_est

array([[2969.890859, 0.000000, -1381.931427, -1778320.891748],
       [0.000000, 2969.890859, 639.427078, 0.000000],
       [0.000000, 0.000000, 1.000000, 0.000000]])

In [ ]:
P_L_est

array([[2969.890859, 0.000000, -1381.931427, 0.000000],
       [0.000000, 2969.890859, 639.427078, 0.000000],
       [0.000000, 0.000000, 1.000000, 0.000000]])

In [ ]:
Trns

array([[-486.148518],
       [7.986519],
       [-349.481345]])

In [ ]:
Rot

array([[0.998009, 0.001370, -0.063064],
       [-0.001795, 0.999976, -0.006681],
       [0.063053, 0.006781, 0.997987]])